<a href="https://colab.research.google.com/github/yo20mom/CUK_Learning/blob/master/DeepLearningNLP/rnn_7th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import keras

from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

In [ ]:
char2vec = {'h': np.array([1,0,0,0,0]),
            'e': np.array([0,1,0,0,0]),
            'l': np.array([0,0,1,0,0]),
            'o': np.array([0,0,0,1,0]),
            '<EOS>': np.array([0,0,0,0,1])}

idx2char = ['h','e','l','o','<EOS>']

In [ ]:
print(char2vec["l"])

In [ ]:
idx2char[2]

In [ ]:
from tensorflow.keras.layers import Embedding

In [ ]:
tf.random.set_seed(1)

inputs = Input(shape=())
embeded = Embedding(input_dim=5, output_dim=2)(inputs)

model = Model(inputs, embeded)

In [ ]:
model.summary()

In [ ]:
model.layers[1].get_weights()

In [ ]:
model.predict([[1]])

In [ ]:
w_xh = np.array([[-2.6, -1.6, -2.1],
                 [1.2, 0.4, 0.3],
                 [2.1, 1.9, -0.7],
                 [-1.4, -1.5, 2.5],
                 [-0.9, 0.4, -0.9]])

w_hh = np.array([[-0.5, -2.3, 2.9],
                 [1.9, 1.5, 1.7],
                 [-0.7, -1.2, 1.5]])

b_h = np.array([-0.5, -0.4, -1.])

w_hy = np.array([[ 0.3, -2.6, 1.2, 2.6, -1.1],
                 [-1.1, -2.4, 2.2, 1.6, -2.4],
                 [-0.4, -3.1, -3., 3.6, 3.]])

b_y = np.array([-1.8, -0.5, 1.3, 0.1, 0.8])


In [ ]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis = -1)

# Time step 1

In [ ]:
init_h = np.array([0,0,0])
x_0 = char2vec['h']
a_1 = np.dot(init_h,w_hh) + np.dot(x_0,w_xh) + b_h
h_1 = np.tanh(a_1)
y_1 = np.dot(h_1, w_hy) + b_y
o_1 = softmax(y_1)


In [ ]:
print("1번째 timestep의 hidden: {}".format(h_1) )
print("1번째 timestep의 output: {}".format(y_1) )
print("1번째 timestep의 result: {}".format(idx2char[np.argmax(o_1)]))

# time step 2

In [ ]:
x_1 = char2vec['e']
a_2 = np.dot(h_1,w_hh) + np.dot(x_1,w_xh) + b_h
h_2 = np.tanh(a_2)
y_2 = np.dot(h_2, w_hy) + b_y
o_2 = softmax(y_2)

In [ ]:
print("2번째 timestep의 hidden: {}".format(h_2) )
print("2번째 timestep의 output: {}".format(y_2) )
print("2번째 timestep의 result: {}".format(idx2char[np.argmax(o_2)]))

# Tensorflow keras 를 이용한 RNN

In [ ]:
from tensorflow.keras.layers import Input, Dense, RNN
from tensorflow.keras.layers import SimpleRNNCell

In [ ]:
n_inputs = 5
n_steps = 5
n_neurons = 3
n_outputs = n_inputs

In [ ]:
inputs = Input(shape=(n_steps, n_inputs))
hidden = RNN(SimpleRNNCell(n_neurons), return_sequences=True)(inputs)
output = Dense(n_outputs, activation= 'softmax')(hidden)


model = Model(inputs, output)

In [ ]:
model.summary()

In [ ]:
model.set_weights([w_xh,w_hh,b_h,w_hy,b_y])

In [ ]:
input_values = "hello"
print("입력값: ", list(input_values))
input_vecs = np.stack([char2vec[char]
                      for char in input_values])[np.newaxis]

result = model.predict(input_vecs)
result_indices = np.argmax(result, axis=-1)[0]
print("출력값: ", [idx2char[idx] for idx in result_indices])

#simple RNN 클래스를 활용한 RNN 모델

In [ ]:
from tensorflow.keras.layers import SimpleRNN

inputs = Input(shape=(n_steps, n_inputs))
hidden = SimpleRNN(n_neurons, return_sequences=True)(inputs)
output = Dense(n_outputs,activation='softmax')(hidden)

In [ ]:
model = Model(inputs, output)

In [ ]:
model.set_weights([w_xh,w_hh,b_h,w_hy,b_y])

In [ ]:
input_values = "hello"
print("입력값: ", list(input_values))
input_vecs = np.stack([char2vec[char]
                      for char in input_values])[np.newaxis]

result = model.predict(input_vecs)
result_indices = np.argmax(result, axis=-1)[0]
print("출력값: ", [idx2char[idx] for idx in result_indices])

In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

In [ ]:
def generate_timeseries(n_steps=50):
    m_x = np.random.uniform(0,10)
    xs = np.linspace(0,5, n_steps+1)
    ys = np.array([0.5*np.sin(2*np.pi*(x+m_x))+ np.cos(3*np.pi/2*(x+m_x/4)) + np.random.uniform(-0.1,0.1) for x in xs])
    return ys[:-1],ys[-1]

In [ ]:
for _ in range(3):
    xs, ys = generate_timeseries(n_steps=40)
    timesteps = np.arange(len(xs)+1)
    plt.figure(figsize=(10,3))
    plt.plot(timesteps[:-1],xs)
    plt.scatter(timesteps[-1],ys,c='r')
    plt.show

In [ ]:
K.clear_session()

In [ ]:
n_inputs = 1
n_steps  = 50
n_neurons = 200
n_outputs = n_inputs

In [ ]:
inputs = Input(shape=(n_steps, n_inputs))
hidden = SimpleRNN(n_neurons,return_state=False)(inputs)
output = Dense(1)(hidden)

In [ ]:
model = Model(inputs, output)

In [ ]:
model.summary()

In [ ]:
model.compile(loss= 'mse', optimizer=Adam(lr=1e-4))

In [ ]:
def timeseries_generator(n_steps = 50, batch_size = 32):
    while True:
        batch_xs, batch_ys = [], []
        for _ in range(batch_size):
            x, y = generate_timeseries(n_steps)
            batch_xs.append(x[:,np.newaxis])
            batch_ys.append(y[np.newaxis])
            yield np.stack(batch_xs), np.stack(batch_ys)

In [ ]:
train_gen = timeseries_generator(batch_size = 3)
batch_x, batch_y = next(train_gen)

for xs, y in zip(batch_x, batch_y):
    plt.plot(xs)
    plt.scatter(len(xs), y, c='r')
    plt.show()

In [ ]:
tarin_gen = timeseries_generator(n_steps, batch_size=16)
hist = model.fit_generator(train_gen,steps_per_epoch= 20, epochs = 20 )

In [ ]:
plt.title("train loss")
plt.plot(hist.history['loss'])
plt.show()

In [ ]:
## 생성된 RNN
for _ in range(3):
    xs, y_true = generate_timeseries(n_steps)
    timesteps = np.arange(len(xs)+1)

    xs = xs[np.newaxis, :, np.newaxis]
    y_pred = model.predict(xs).squeeze()

    plt.figure(figsize=(10,3))
    plt.plot(timesteps[:-1],xs.squeeze())


    plt.scatter(timesteps[-1],y_true,c='g', marker = '+')
    plt.scatter(timesteps[-1],y_pred,c='r', marker = '*')
    plt.show()